# Antrenarea U-Net

### Import:

In [1]:
import os   
import numpy as np    # pentru a se putea utiliza funcționalități pe imagini
import cv2            #read, resize and write images
from glob import glob # extract file based on a specific pattern
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.layers import Input

### Pentru sedimentare (seeding): 

In [2]:
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

### Setarea Hiperparametrilor

In [3]:
batch_size = 8  # deoarece am 8gb de memorie
lr = 1e-4    #learning rate
epochs = 100
height = 768
width = 512


### Calea (Path):

In [4]:
dataset_path = os.path.join("dataset", "non-aug")

files_dir = os.path.join("files", "non-aug")
model_file = os.path.join(files_dir, "unet-non-aug.h5")
log_file = os.path.join(files_dir, "log-non-aug.csv")

### Crearea Fișierelor

In [5]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [6]:
create_dir(files_dir)

### Construirea U-Net

#### Construirea blocului de convoluție

In [20]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    return x

#### Construirea blocului encoder

In [8]:
def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    
    return x, p

#### Construirea blocului decoder

In [9]:
def decoder_block(inputs, skip, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip])
    x = conv_block(x, num_filters)
    return x

#### UNET

In [10]:
def build_unet(input_shape):
    inputs = Input(input_shape)
    
    """ Encoder """
    
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)
    
    """ Bridge """
    
    b1 = conv_block(p4, 1024)
    
    """ Decoder """
    
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
    
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)
    
    model = Model(inputs, outputs, name="UNET")
    
    return model


### Dataset Pipeline

#### încărcarea setului de date de antrenare și validare:

In [11]:
def load_data(path):
    train_x = sorted(glob(os.path.join(path, "train", "images", "*")))
    train_y = sorted(glob(os.path.join(path, "train", "masks", "*")))
    
    valid_x = sorted(glob(os.path.join(path, "valid", "images", "*")))
    valid_y = sorted(glob(os.path.join(path, "valid", "masks", "*")))
    
    return (train_x, train_y), (valid_x, valid_y)


#### Citirea imaginilor:

In [12]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = x/255.0
    return x

#### Citirea măștilor:

In [13]:
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = x/255.0
    x = np.expand_dims(x, axis=-1)
    return x


#### tf.data Pipeline

In [14]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y 
    
    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
    x.set_shape([height, width, 3])
    y.set_shape([height, width, 1])
    
    return x, y

In [15]:
def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

### Antrenarea:

In [16]:
(train_x, train_y), (valid_x, valid_y) = load_data(dataset_path)
print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

Train: 60 - 60
Valid: 20 - 20


In [17]:
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

In [18]:
for x, y in valid_dataset:
    print(x.shape, y.shape)

(8, 768, 512, 3) (8, 768, 512, 1)
(8, 768, 512, 3) (8, 768, 512, 1)
(4, 768, 512, 3) (4, 768, 512, 1)


In [21]:
input_shape = (height, width, 3)
model = build_unet(input_shape)

In [22]:
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 768, 512, 3)]        0         []                            
                                                                                                  
 conv2d_1 (Conv2D)           (None, 768, 512, 64)         1792      ['input_2[0][0]']             
                                                                                                  
 batch_normalization_1 (Bat  (None, 768, 512, 64)         256       ['conv2d_1[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 activation_1 (Activation)   (None, 768, 512, 64)         0         ['batch_normalization_1[0][

 batch_normalization_9 (Bat  (None, 48, 32, 1024)         4096      ['conv2d_9[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 activation_9 (Activation)   (None, 48, 32, 1024)         0         ['batch_normalization_9[0][0]'
                                                                    ]                             
                                                                                                  
 conv2d_10 (Conv2D)          (None, 48, 32, 1024)         9438208   ['activation_9[0][0]']        
                                                                                                  
 batch_normalization_10 (Ba  (None, 48, 32, 1024)         4096      ['conv2d_10[0][0]']           
 tchNormalization)                                                                                
          

 concatenate_3 (Concatenate  (None, 768, 512, 128)        0         ['conv2d_transpose_3[0][0]',  
 )                                                                   'activation_2[0][0]']        
                                                                                                  
 conv2d_17 (Conv2D)          (None, 768, 512, 64)         73792     ['concatenate_3[0][0]']       
                                                                                                  
 batch_normalization_17 (Ba  (None, 768, 512, 64)         256       ['conv2d_17[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_17 (Activation)  (None, 768, 512, 64)         0         ['batch_normalization_17[0][0]
                                                                    ']                            
          

In [24]:
opt = tf.keras.optimizers.Adam(lr)
model.compile(loss="binary_crossentropy",optimizer=opt, metrics=["acc"])


In [25]:
callbacks = [
    ModelCheckpoint(model_file, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
    CSVLogger(log_file),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)

 ]

In [ ]:
model.fit(
train_dataset,
validation_data=valid_dataset,
epochs=epochs,
callbacks=callbacks
)

Epoch 1/100
8/8 [==============================] - ETA: 0s - loss: 0.7054 - acc: 0.5508  
Epoch 1: val_loss improved from inf to 0.69555, saving model to files/non-aug/unet-non-aug.h5


/Users/taras_lara/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 1998s 251s/step - loss: 0.7054 - acc: 0.5508 - val_loss: 0.6955 - val_acc: 0.3481 - lr: 1.0000e-04
Epoch 2/100
8/8 [==============================] - ETA: 0s - loss: 0.6026 - acc: 0.7074  
Epoch 2: val_loss did not improve from 0.69555
8/8 [==============================] - 1873s 230s/step - loss: 0.6026 - acc: 0.7074 - val_loss: 0.6999 - val_acc: 0.2769 - lr: 1.0000e-04
Epoch 3/100
8/8 [==============================] - ETA: 0s - loss: 0.5255 - acc: 0.7650   
Epoch 3: val_loss did not improve from 0.69555
8/8 [==============================] - 10708s 1405s/step - loss: 0.5255 - acc: 0.7650 - val_loss: 0.7003 - val_acc: 0.3379 - lr: 1.0000e-04
Epoch 4/100
8/8 [==============================] - ETA: 0s - loss: 0.4559 - acc: 0.8198  
Epoch 4: val_loss did not improve from 0.69555
8/8 [==============================] - 1847s 226s/step - loss: 0.4559 - acc: 0.8198 - val_loss: 0.6959 - val_acc: 0.4833 - lr: 1.0000e-04
Epoch 5/100
8/8 [=================